## TFRecord ver 2

In [22]:
import os
import tensorflow as tf
import json

def json_to_tfexample(json_data, img_path):

    # JSON 데이터 파싱
    metadata = json_data['metaData']
    labeling_info = json_data['labelingInfo']

    # TFExample에 넣을 features 설정
    features = {
        'Raw_data_ID': tf.train.Feature(bytes_list=tf.train.BytesList(value=[metadata['Raw data ID'].encode('utf-8')])),
        'breed': tf.train.Feature(bytes_list=tf.train.BytesList(value=[metadata['breed'].encode('utf-8')])),
        'age': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(metadata['age'])])),
        'gender': tf.train.Feature(bytes_list=tf.train.BytesList(value=[metadata['gender'].encode('utf-8')])),
        'species': tf.train.Feature(bytes_list=tf.train.BytesList(value=[metadata['species'].encode('utf-8')])),
        'lesions': tf.train.Feature(bytes_list=tf.train.BytesList(value=[metadata['lesions'].encode('utf-8')])),
        'path': tf.train.Feature(bytes_list=tf.train.BytesList(value=[metadata['Path'].encode('utf-8')])),
        'img_path': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_path.encode('utf-8')])),
        # 'src_path': tf.train.Feature(bytes_list=tf.train.BytesList(value=[metadata['src_path'].encode('utf-8')]))
    }

    # Polygon 정보 추출
    polygon_location = labeling_info[0]['polygon']['location'][0]
    # x, y 좌표를 따로 리스트로 저장
    x_coords = [polygon_location[f'x{i}'] for i in range(1, len(polygon_location) // 2 + 1)]
    y_coords = [polygon_location[f'y{i}'] for i in range(1, len(polygon_location) // 2 + 1)]
    # TFRecord의 feature 업데이트 (VarLenFeature로 저장)
    polygon_features = {
        'x_coords': tf.train.Feature(int64_list=tf.train.Int64List(value=x_coords)),
        'y_coords': tf.train.Feature(int64_list=tf.train.Int64List(value=y_coords))
    }

    # Label 업데이트
    label = labeling_info[0]['polygon']['label']
    features.update(polygon_features)
    features['label'] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[label.encode('utf-8')]))

    # Box 정보 추출
    if any('box' in info for info in labeling_info):
        for info in labeling_info:
            if 'box' in info:
                box_data = info['box']
                box_location = box_data['location'][0] if box_data['location'] else {}
                box_features = {
                    'box_x': tf.train.Feature(int64_list=tf.train.Int64List(value=[box_location.get('x', -1)])),
                    'box_y': tf.train.Feature(int64_list=tf.train.Int64List(value=[box_location.get('y', -1)])),
                    'box_width': tf.train.Feature(int64_list=tf.train.Int64List(value=[box_location.get('width', -1)])),
                    'box_height': tf.train.Feature(int64_list=tf.train.Int64List(value=[box_location.get('height', -1)]))
                }
                features.update(box_features)
                features['box_label'] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[box_data.get('label', '').encode('utf-8')]))
    else:
        print(f"Box 정보가 없는 JSON 파일: {metadata['Raw data ID']}")
    example = tf.train.Example(features=tf.train.Features(feature=features))
    return example




# TFRecord 파일로 변환하는 함수
def convert_to_tfrecord(folder_path, output_file):
    json_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))

    with tf.io.TFRecordWriter(output_file) as writer:
        for json_file in json_files:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # 이미지 경로 지정
            img_path = '../Segmentation/train_labelingdata/dog/skin/camera/symp/A3'
            example = json_to_tfexample(data, img_path)
            writer.write(example.SerializeToString())

# JSON 파일이 있는 폴더 경로
folder_path = r'D:\Final_PJT\Segmentation\train_labeling_data\dog\skin\camera\symp\A3'

# TFRecord 파일 경로
output_file = 'train_symptoms_A3.tfrecord'

# TFRecord 파일로 변환
convert_to_tfrecord(folder_path, output_file)

print(f'TFRecord 파일이 생성되었습니다: {output_file}')


TFRecord 파일이 생성되었습니다: train_symptoms_A3.tfrecord


## TFRecord --> DataFrame 으로 변경 후 저장

In [23]:
import pandas as pd
import tensorflow as tf

# TFRecord 파일로부터 데이터를 읽어 데이터 프레임으로 만드는 함수
def parse_tfrecord(record):
    feature_description = {
        'Raw_data_ID': tf.io.FixedLenFeature([], tf.string),
        'breed': tf.io.FixedLenFeature([], tf.string),
        'age': tf.io.FixedLenFeature([], tf.int64),
        'gender': tf.io.FixedLenFeature([], tf.string),
        'species': tf.io.FixedLenFeature([], tf.string),
        'lesions': tf.io.FixedLenFeature([], tf.string),
        'path': tf.io.FixedLenFeature([], tf.string),
        'img_path': tf.io.FixedLenFeature([], tf.string),
        'x_coords': tf.io.VarLenFeature(tf.int64),
        'y_coords': tf.io.VarLenFeature(tf.int64),
        'label': tf.io.FixedLenFeature([], tf.string),
        'box_x': tf.io.FixedLenFeature([], tf.int64, default_value=0),  # 기본값으로 0 지정
        'box_y': tf.io.FixedLenFeature([], tf.int64, default_value=0),  # 기본값으로 0 지정
        'box_width': tf.io.FixedLenFeature([], tf.int64, default_value=0),  # 기본값으로 0 지정
        'box_height': tf.io.FixedLenFeature([], tf.int64, default_value=0),  # 기본값으로 0 지정
        'box_label': tf.io.FixedLenFeature([], tf.string, default_value=""),  # 기본값으로 빈 문자열 지정
    }
    example = tf.io.parse_single_example(record, feature_description)
    return example

# TFRecord 파일을 데이터프레임으로 변환하는 함수
def tfrecord_to_dataframe(tfrecord_file):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    parsed_dataset = dataset.map(parse_tfrecord)
    
    records = []
    for record in parsed_dataset:
        record_dict = {}
        for key, value in record.items():
            if isinstance(value, tf.sparse.SparseTensor):
                record_dict[key] = value.values.numpy()
            else:
                record_dict[key] = value.numpy()
        records.append(record_dict)
    
    return pd.DataFrame.from_records(records)

# TFRecord 파일 경로
tfrecord_file = 'train_symptoms_A3.tfrecord'

# TFRecord 파일을 데이터프레임으로 변환
df = tfrecord_to_dataframe(tfrecord_file)

# 데이터프레임을 xlsx 파일로 저장
excel_file = 'train_symptoms_A3.xlsx'
df.to_excel(excel_file, index=False)

print(f'DataFrame을 EXCEL 파일로 저장하였습니다: {excel_file}')


DataFrame을 EXCEL 파일로 저장하였습니다: train_symptoms_A3.xlsx
